In [1]:
import numpy as np
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [2]:
df_data=pd.read_csv('train.csv',encoding='utf-8')
df_label=pd.read_csv('train_target.csv',encoding='utf-8')
df_test=pd.read_csv('test.csv',encoding='utf-8')

In [3]:
X = df_data.drop(['id','isNew'], axis=1)
Y = df_label['target']
Z = df_test.drop(['id','isNew'], axis=1)

In [4]:
#X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=.2,random_state=0)
X_train,Y_train = X,Y

In [28]:
model = xgb.XGBClassifier(base_score = 0.06)
model.fit(X_train,Y_train)

XGBClassifier(base_score=0.06, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [29]:
y_pred = model.predict_proba(X_train)[:,1]
test_pred = model.predict_proba(Z)[:,1]
# predictions = [round(value) for value in test_pred]
test_pred

array([0.00641354, 0.01419097, 0.00584857, ..., 0.00755358, 0.00490803,
       0.00346874], dtype=float32)

In [30]:
auc = roc_auc_score(Y_train.values, y_pred)
print(auc)

0.7908901769688534


In [32]:
sub = pd.read_csv('submission.csv',encoding='utf-8')
p = np.array(test_pred).reshape(-1,1)
sub['target'] = p

In [33]:
sub[['id','target']].to_csv('sub.csv',index=False)

线下AUC随base_score变化：
1      0.77
0.07 0.7872
0.06 0.7908
0.05 0.7897